In [14]:
import pandas as pd
import geopandas as gpd
import altair as alt
import numpy as np
import os
import chromatose as ct

import geopandas as gpd
import json

from IPython.display import SVG
alt.renderers.set_embed_options(actions=False, renderer='svg')

RendererRegistry.enable('default')

In [19]:
df_model_ij = pd.read_csv('processed_data/DF_MODEL_IJ.csv')
df_model_i = pd.read_csv('processed_data/DF_MODEL_I.csv')
df_model_j = pd.read_csv('processed_data/DF_MODEL_J.csv')

In [15]:
df_compare_ij = pd.read_csv('processed_data/DF_COMPARE_IJ.csv')
df_compare_i = pd.read_csv('processed_data/DF_COMPARE_I.csv')
df_compare_j = pd.read_csv('processed_data/DF_COMPARE_J.csv')

In [16]:
# used for centroid calculation, simplify used for plot
state_tracts = gpd.read_file(
    'processed_data/california_tracts_2020.geojson')

state_tracts_simplify = gpd.read_file(
    'processed_data/california_tracts_2020_simplify.geojson')
state_tracts_simplify = state_tracts_simplify.to_crs(epsg=4326)

state_counties = gpd.read_file('processed_data/california_counties_2020.geojson')
state_counties = state_counties.to_crs(epsg=4326)

pali_fire = gpd.read_file('processed_data/palisades_fire_perimeter.geojson')
eaton_fire = gpd.read_file('processed_data/eaton_fire_perimeter.geojson')
pali_fire = pali_fire.set_crs(epsg=3857, allow_override=True).to_crs(epsg=4326)
eaton_fire = eaton_fire.set_crs(epsg=3857, allow_override=True).to_crs(epsg=4326)

### Lij from data

In [20]:
df_model_ij.columns

Index(['county_i', 'county_j', 'county_i_name', 'county_j_name', 'L_ij_data'], dtype='object')

In [27]:
def plot_L_ij_data(df_model_ij, scale='symlog'):
    palette = ct.palpolate(ct.nacre[:-1])[::-1]
    
    heatmap = (alt.Chart(df_model_ij).mark_rect().encode(
            x=alt.X("county_j:N", title="Work county", axis=alt.Axis(labelAngle=45)),
            y=alt.Y("county_i:N", title="Resident county"),
            color=alt.Color(
                "L_ij_data:Q",
                title="Number of workers",
                scale=alt.Scale(
                    type=scale, 
                    range=palette
                )  # important for skewed flows
            ),
            tooltip=[
                "county_i:N",
                "county_j:N",
                "L_ij_data:Q"
            ]
        )
        .properties(
            width=300,
            height=300,
            title=f"Commuting flows: residence → workplace ({scale} color)"
        )
    )
    
    outside_share = (
        df_model_ij
        .assign(outside=lambda d: d["county_j"] != d["county_i"])
        .groupby("county_i")
        .apply(lambda g: g.loc[g.outside, "L_ij_data"].sum() / g["L_ij_data"].sum())
        .reset_index(name="pct_outside")
    )
    
    outside_share["label"] = (outside_share["pct_outside"] * 100).round(1).astype(str) + "%"
    
    text = (
        alt.Chart(outside_share)
        .mark_text(align="left", dx=5)
        .encode(
            y=alt.Y("county_i:N"),
            x=alt.value(400),   # slightly to the right of heatmap width
            text="label:N"
        )
    ) 
    
    # -----------------------------
    outside_share_cell = (
        df_model_ij
        .assign(outside=lambda d: d["county_i"] != d["county_j"])
        .groupby("county_j")
        .apply(lambda g: g.loc[g.outside, "L_ij_data"].sum() / g["L_ij_data"].sum())
        .reset_index(name="pct_outside")
    )
    
    outside_share_cell["label"] = (outside_share_cell["pct_outside"] * 100).round(1).astype(str) + "%"
    
    text_2 = (
        alt.Chart(df_model_ij)
        .transform_lookup(
            lookup="county_i",
            from_=alt.LookupData(
                outside_share_cell,
                key="county_i",
                fields=["label"]
            )
        )
        .mark_text(color="black", fontSize=9)
        .encode(
            x="county_j:N",
            y="county_i:N",
            text="label:N"
        )
    )
    return heatmap + text

In [29]:
chart_Lij_data_lin = plot_L_ij_data(df_model_ij, scale='linear')
chart_Lij_data_log = plot_L_ij_data(df_model_ij, scale='symlog')
alt.hconcat(chart_Lij_data_lin, chart_Lij_data_log).resolve_scale(color='independent')

/var/folders/6y/m69ngqb568nbb5t5p3lz216r0000gn/T/ipykernel_26177/722867631.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_model_ij
/var/folders/6y/m69ngqb568nbb5t5p3lz216r0000gn/T/ipykernel_26177/722867631.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_model_ij
/var/folders/6y/m69ngqb568nbb5t5p3lz216r0000gn/T/ipykernel_26177/722867631.py:29: DeprecationWarning: DataFrameGroupBy.apply opera

alt.HConcatChart(...)

### Counterfactual L_ij - prediction L_ij 

Note that there are three different L_ij columns: counterfactual, prediction, and data.

In [34]:
def plot_L_ij_compare(df_compare_ij, scale='symlog'):
    # df_compare_ij = df_compare_ij.merge(df_compare_i[['county_i_name', 'L_i_data']], how='left', on='county_i_name')
    # # proportion of change in L_ij over 
    # df_compare_ij['diff_L_ij_prop'] = df_compare_ij['diff_L_ij'] / df_compare_ij['L_i_data']
    
    # palette = ct.palpolate(ct.nacre[:-1])[::-1]
    max_val = df_compare_ij['diff_L_ij'].abs().max() 
    heatmap_diff_counts = alt.Chart(df_compare_ij).mark_rect(stroke='#cdcdcd', strokeWidth=1).encode(
            x=alt.X("county_i_name:N", title="Resident county", axis=alt.Axis(labelAngle=45)),
            y=alt.Y("county_j_name:N", title="Work county"),
            color=alt.Color(
                "diff_L_ij:Q",
                title="Δ in # of workers (in thousands)",
                scale=alt.Scale(
                    scheme='redblue',
                    type=scale, 
                    domain=[-1e3, 1e3], clamp=True)  # important for skewed flows
            ), tooltip=["county_i_name:N","county_j_name:N","diff_L_ij:Q"]
        ).properties(width=300, height=300, title=[f"Δ in commuting flowsin thousands:", f"cf - eq. in thousands ({scale} color)"])
    
    
    return heatmap_diff_counts

In [35]:
chart_Lij_compare_lin = plot_L_ij_compare(df_compare_ij, scale='linear')
chart_Lij_compare_log = plot_L_ij_compare(df_compare_ij, scale='symlog')
alt.hconcat(chart_Lij_compare_lin, chart_Lij_compare_log).resolve_scale(color='independent')

alt.HConcatChart(...)

The heatmap above shows the net outflow from workerse in LA, who after the shock go on to work in Ventura and Orange County.
We see a strong diagonal bc even small changes in $\pi_{ij}$ multiplied by large resident populations produce a big $\Delta L_{ii}$

### $p_i$ at equilibrium, counterfactual, and difference

In [37]:
d_county_FID = {
    'Los Angeles':1993, 
    'Orange':362, 
    'Riverside':2943, 
    'San Bernardino':817,
    'San Diego':1896, 
    'Ventura':963
}

df_compare_i['FID'] = df_compare_i['county_i_name'].map(d_county_FID)
df_compare_j['FID'] = df_compare_j['county_j_name'].map(d_county_FID)

df_compare_i = df_compare_i.merge(state_counties, how='left', on='FID')
df_compare_j = df_compare_j.merge(state_counties, how='left', on='FID')

In [38]:
# geojson = json.loads(df_compare_i.to_json())
geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": geom.__geo_interface__,
            "properties": row.drop("geometry").to_dict(),
        }
        for _, row in df_compare_i.iterrows()
        for geom in [row.geometry]
    ]
}

In [46]:
map_p_i_eq = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape(stroke='#cdcdcd', strokeWidth=1.5)
    .encode(
        color=alt.Color(
            "properties.p_i_eq:Q", title="p_i (eq)",
            scale=alt.Scale(scheme='purples'))
    ).project(type="mercator"
    ).properties(width=250, height=250, title='p_i EQ')
)

map_p_i_cf = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape(stroke='#ababab', strokeWidth=1.5)
    .encode(
        color=alt.Color(
            "properties.p_i_cf:Q", title="p_i (cf)",
            scale=alt.Scale(scheme='purples'))
    ).project(type="mercator"
    ).properties(width=250, height=250, title='p_i CF')
)

map_p_i_diff = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape(stroke='#ababab', strokeWidth=1.5)
    .encode(
        color=alt.Color(
            "properties.diff_p_i:Q", title="Δ p_i", 
            scale=alt.Scale(scheme='redblue', reverse=True, domainMid=0))
    ).project(type="mercator"
    ).properties(width=250, height=250, title='Δ p_i, CF - EQ')
)

In [47]:
alt.hconcat(map_p_i_eq, map_p_i_cf)

alt.HConcatChart(...)

In [48]:
map_p_i_diff

alt.Chart(...)

Prices go up in LA, and especially in neighboring Ventura county (left) and Orange County (right). We see that Riverside and San Diego are less affected.

### Plot L_j_eq, L_j_cf , change in L_j

Now let's look at workers in county j.

In [11]:
# geojson = json.loads(df_compare_i.to_json())
geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": geom.__geo_interface__,
            "properties": row.drop("geometry").to_dict(),
        }
        for _, row in df_compare_j.iterrows()
        for geom in [row.geometry]
    ]
}

map_L_j_eq = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape(stroke='#cdcdcd', strokeWidth=1.5)
    .encode(
        color=alt.Color(
            "properties.L_j_eq:Q", title="L_j (eq)",
            scale=alt.Scale(scheme='reds'))
    ).project(type="mercator"
    ).properties(width=250, height=250, title='L_j EQ')
)

map_L_j_cf = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape(stroke='#ababab', strokeWidth=1.5)
    .encode(
        color=alt.Color(
            "properties.L_j_cf:Q", title="L_j (cf)",
            scale=alt.Scale(scheme='reds'))
    ).project(type="mercator"
    ).properties(width=250, height=250, title='L_j CF')
)

map_L_j_diff = (
    alt.Chart(alt.Data(values=geojson["features"]))
    .mark_geoshape(stroke='#ababab', strokeWidth=1.5)
    .encode(
        color=alt.Color(
            "properties.diff_L_j:Q", title="Δ L_j", 
            scale=alt.Scale(scheme='redblue', domainMid=0))
    ).project(type="mercator"
    ).properties(width=250, height=250, title='Δ p_i, CF - EQ')
)

In [12]:
alt.hconcat(map_L_j_eq, map_L_j_cf)

alt.HConcatChart(...)

In [13]:
map_L_j_diff

alt.Chart(...)